In [193]:
import pandas as pd
import  os
import numpy as np
import psycopg2
import glob
import openpyxl

In [211]:
df = pd.read_excel("chart_of_Accounts.xlsx")
df.head()

,Account ID,Account Name,Account Code,Description,Account Type,Mileage Rate,Mileage Unit,IsMileage,Account #,Currency,Parent Account
0,2620362000000010001,Other Charges,223500001.0,Miscellaneous charges like adjustments made to...,Income,0.0,NaN,False,NaN,GHS,NaN
1,2620362000000032023,Lodging,223400001.0,Any expense related to putting up at motels et...,Expense,0.0,NaN,False,NaN,GHS,NaN
2,2620362000000000358,Undeposited Funds,111000001.0,Record funds received by your company yet to b...,Cash,0.0,NaN,False,NaN,GHS,NaN
3,2620362000000000361,Petty Cash,111000002.0,It is a small amount of cash that is used to p...,Cash,0.0,NaN,False,NaN,GHS,Undeposited Funds
4,2620362000000078007,MTN Mobile Money,111000003.0,NaN,Cash,0.0,NaN,False,NaN,GHS,Undeposited Funds


In [212]:
df.columns = [x.lower().replace("#","number").replace(" ","_").replace("%","").replace(".","") for x in df.columns]
df.columns

Index(['account_id', 'account_name', 'account_code', 'description',
       'account_type', 'mileage_rate', 'mileage_unit', 'ismileage',
       'account_number', 'currency', 'parent_account'],
      dtype='object')

In [213]:
df.dtypes

account_id          int64
account_name       object
account_code      float64
description        object
account_type       object
mileage_rate      float64
mileage_unit      float64
ismileage            bool
account_number    float64
currency           object
parent_account     object
dtype: object

In [216]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float',
    'int64' : 'varchar',
    'datetime64[ns]': 'varchar',
    'datetime64': 'timestamp',
    'bool' : 'varchar'
}
replacements

{'object': 'varchar',
 'float64': 'float',
 'int64': 'varchar',
 'datetime64[ns]': 'varchar',
 'datetime64': 'timestamp',
 'bool': 'varchar'}

In [217]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'account_id varchar, account_name varchar, account_code float, description varchar, account_type varchar, mileage_rate float, mileage_unit float, ismileage varchar, account_number float, currency varchar, parent_account varchar'

In [218]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [219]:
cursor.execute("drop table if exists account_details;")

In [220]:
cursor.execute("create table account_details \
    (account_id varchar, account_name varchar, account_code varchar, description varchar, account_type varchar, mileage_rate float, mileage_unit float, ismileage varchar, account_number varchar, currency varchar, parent_account varchar)")

In [221]:
#insert values to tables
#save df to csv
df.to_csv('account_details', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("account_details")
print('file opened in memory')

file opened in memory


In [222]:
SQL_STATEMENT = """
COPY account_details FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [223]:
cursor.execute("grant select on table account_details to public")
conn.commit()

cursor.close()
print('table account_details imported successfully')

table account_details imported successfully
